In [ ]:
import os
import requests
from bs4 import BeautifulSoup
import json
from lxml import etree
import pandas as pd
import logging
import time
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s', force=True)

## Come installare la cartella con tutti i documenti

1.   Vai su Google Drive.
2.   Trova la cartella condivisa "progettoIngDati".
3.   Fai clic con il pulsante destro del mouse sulla cartella e   
     seleziona :
     - Organizza.
     - Aggiungi Scorciatoia
     - Tutte le posizioni
4.   Scegli "My Drive" come destinazione per il collegamento.

In [41]:
#in questo modo caricate il vostro drive, che peró noi non vedremo!
#ma prima é importante che eseguiate le istruzioni qui sopra!
from google.colab import drive
drive.mount('/content/drive', force_remount=True)


articoli_path = '/content/drive/MyDrive/progettoIngDati/sources'

extraction_path = '/content/drive/MyDrive/progettoIngDati/extraction'

if os.path.exists(articoli_path):
    logging.info("La cartella articoli esiste!")
else:
    logging.error("Impossibile trovare la cartella articoli.")

2024-10-15 16:15:50,211 - INFO - La cartella articoli esiste!


Mounted at /content/drive


## Ogni volta che ci serve tutta la cartella basta copiare questa cella di codice e inserirla all'inizio del colab dove ci servono gli articoli! Successivamente avremo path

Codice da chat, non rivisto, avevo trovato un html che scaricato era scritto strano, come se alcuni campi fossero sovrapposti, non so se mi si era buggato a me, in ongni caso potremmo fare quanche controllo su altri documenti.

Cancella la cartella articoli/ (se  serve)

In [ ]:
!rm -rf articoli/

# DOI Scouting (article id) and HTML Download

In [ ]:
TOTAL_DOCUMENTS_NUMBER = 595        # visit url page
article_ids = set()
TOPIC = "LLM multi agent"

Scarica il file HTML5 da ar5iv

In [ ]:
def get_ar5iv_html(article_id):
    urls = [ f"https://ar5iv.labs.arxiv.org/html/{article_id}",
             f"https://arxiv.org/html/{article_id}" ]               # sometimes html is provided on this url when it's not showing on ar5ive
    found_urls = set()
    file_path = '/content/drive/MyDrive/progettoIngDati/read.me'
    for url in urls:
        try:
            response = requests.get(url, timeout=10)
            logging.info(f"Request URL: {response.url}")

            if response.status_code == 200:
                soup = BeautifulSoup(response.text, 'html.parser')

                # Trova il tag <article> o altri contenuti rilevanti
                article = soup.find('article')

                if article:
                    found_urls.add(url)
                    return article

                # Se non troviamo un tag <article>, proviamo con <div> o <section>
                # if not article:
                #     article = soup.find('div', {'class': 'ltx_page'})  # Prova a trovare un'altra sezione

            else:
                logging.error(f"Error fetching article {article_id}, status code: {response.status_code}")
                return None

        except requests.exceptions.RequestException as e:
            logging.error(f"Error downloading article {article_id}: {e}")
            return None
    with open(file_path, 'w') as f:
        f.writeLines([url + '\n' for url in found_urls])
        logging.info(f"Successfully wrote {len(found_urls)} urls to {file_path}")



Salva l'articolo in un file


In [ ]:
def save_to_file(article, article_id):
    # Crea una cartella per gli articoli se non esiste
    #os.makedirs("articoli", exist_ok=True)
    #file_path = os.path.join("articoli", f"ar5iv_article_{article_id}.html")
    #with open(file_path, "w", encoding="utf-8") as file:
    #    file.write(article.prettify())
    #se hai montato la cartella questo funziona, altrimenti usa la parte commentata qui sopra
    file_path = os.path.join(articoli_path, f"ar5iv_article_{article_id}.html") #join with articoli_path
    with open(file_path, "w", encoding="utf-8") as file:
        file.write(article.prettify())
    logging.info(f"Articolo {article_id} salvato.")

Cerca articoli simili e prova a scaricarne il contenuto html

In [ ]:
def retrieve_article_id(url, params):
    response = requests.get(url, params=params)
    logging.info(f"Request URL: {response.url}")

    if response.status_code == 200:
        soup = BeautifulSoup(response.text, "html.parser")

        ol_tag = soup.find('ol')
        if ol_tag:
            li_tags = ol_tag.find_all('li')
            local_article_counter = 0

            for li in li_tags:
                a_tag = li.find('a')
                if a_tag:
                    article_id = a_tag.text.split(':')[-1]
                    logging.info(f"Processing article number: {len(article_ids)}: {article_id}")
                    article = get_ar5iv_html(article_id)
                    if article:
                        logging.info(f"Article found: {article_id}")
                        local_article_counter += 1
                        article_ids.add(article_id)
                        save_to_file(article, article_id)
                    else:
                        logging.warning(f"Article {article_id} not found, skipping...")
                else:
                    logging.warning("No <a> element found in the <li> element.")

            logging.info(f"Number of local articles found: {local_article_counter} over 200")  # inside a single page
            time.sleep(2)
            return local_article_counter
        else:
            logging.warning("No <ol> element found.")
            return None
    else:
        logging.error(f"Failed to retrieve data. Status code: {response.status_code}")
        return None


In [ ]:
# Define the URL for the advanced search on arXiv
url = "https://arxiv.org/search/advanced"

# Define the search parameters
params = {
    "advanced": "",
    "terms-0-term": TOPIC,              # Search term
    "terms-0-operator": "AND",          # Operator
    "terms-0-field": "all",             # Search field (all)
    "classification-computer_science": "y",  # Limit to computer science
    "classification-physics_archives": "all",  # Include all physics archives
    "classification-include_cross_list": "include",  # Include cross-list
    "date-filter_by": "all_dates",      # Filter by date (all dates)
    "date-year": "",                    # No specific year
    "date-from_date": "",               # No start date
    "date-to_date": "",                 # No end date
    "date-date_type": "submitted_date", # Search by submission date
    "abstracts": "hide",                # Show abstracts
    "size": 200,                        # Number of results to display
    "order": "submitted_date"           # Order by submission date (oldest first) #older documents are more likely to have html version
}

doi_retrieved = 0
while doi_retrieved <= 300:
    doi_retrieved += retrieve_article_id(url, params)
    params["start"] = doi_retrieved

logging.info(f"Number of articles retrieved: {len(article_ids)}")


2024-10-15 16:09:37,259 - INFO - Request URL: https://arxiv.org/search/advanced?advanced=&terms-0-term=LLM+multi+agent&terms-0-operator=AND&terms-0-field=all&classification-computer_science=y&classification-physics_archives=all&classification-include_cross_list=include&date-filter_by=all_dates&date-year=&date-from_date=&date-to_date=&date-date_type=submitted_date&abstracts=hide&size=200&order=submitted_date
2024-10-15 16:09:37,726 - INFO - Processing article number: 0: 2208.13266
2024-10-15 16:09:38,026 - INFO - Request URL: https://ar5iv.labs.arxiv.org/html/2208.13266
2024-10-15 16:09:39,210 - INFO - Article found: 2208.13266
2024-10-15 16:09:39,729 - INFO - Articolo 2208.13266 salvato.
2024-10-15 16:09:39,733 - INFO - Processing article number: 1: 2305.08291
2024-10-15 16:09:40,153 - INFO - Request URL: https://ar5iv.labs.arxiv.org/html/2305.08291
2024-10-15 16:09:40,408 - INFO - Article found: 2305.08291
2024-10-15 16:09:40,554 - INFO - Articolo 2305.08291 salvato.
2024-10-15 16:09:

In [ ]:
print(len(found_urls))

NameError: name 'found_urls' is not defined

# Data Extraction

In [ ]:
def estrai_dati_da_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        content = f.read()

    parser = etree.HTMLParser()
    tree = etree.fromstring(content, parser)

    tabelle = {}
    table_counter = 0

    figure_with_tables = tree.xpath('//figure[.//table]')
  	#per l'articolo 2277 questo non funziona perché i dati della tabella non sono in table ma in diversi span, peró ci sta comunque il figcaption



    for figure in figure_with_tables:
        try:
            table_id = figure.xpath("@id")[0]
            #print("table_id:", table_id)
            table = figure.xpath('.//table')[0]
            table_counter += 1
            table_key = f"id_table_{table_counter}"

            caption = figure.xpath('.//figcaption//text()')
            caption_text = ""  # Initialize caption_text

            if caption:
                caption_text = ' '.join([c.strip() for c in caption]).replace('  ', ' ')


            dati_tabella = []
            rows = table.xpath('.//tr[position()>1]')
            for row in rows:
                cols = row.xpath('.//td')
                dati_row = [etree.tostring(col, encoding='unicode', method='html') for col in cols]
                dati_tabella.append(dati_row)



            references = tree.xpath(f"//p[a/@href = '#{table_id}']")
            references_text = [ref.xpath('string(.)').replace('\n', '').strip() for ref in references]   #elimina /n ma introduce spazi bianchi
            #print(references_text)

            note_a_pie_di_pagina = []
            footnotes = tree.xpath(".//span[contains(@id, 'footnote') and contains(@class, 'ltx_text')]")
            for footnote in footnotes:
              footnote_text = " ".join(footnote.xpath(".//text()")).strip()
              if footnote_text and footnote_text != " ":
                footnote_text = footnote_text.replace("\n", " ")
                note_a_pie_di_pagina.append(footnote_text)
            #logging.info(note_a_pie_di_pagina)


            tabelle[table_key] = {
                "caption": caption_text,
                "table": dati_tabella,
                "footnotes": note_a_pie_di_pagina,
                "references": references_text,
            }


        except Exception as e:
            logging.error(f"Error processing figure in {file_path}: {e}")
            logging.error(f"Figure content: {etree.tostring(figure, encoding='unicode', pretty_print=True)}")  # Print figure content for debugging
            # You might want to add a break statement here if you want to stop processing
            # the file after encountering an error:
            # break

    for table_key, table_data in tabelle.items():
        logging.info(f"Footnotes for table {table_key}: {table_data['footnotes']}")
    return tabelle

In [ ]:
# Funzione per salvare i dati estratti in un file JSON
def salva_dati_in_json(dati_estratti, article_id, extraction_path):
    tabelle = dati_estratti

    # Salva i dati in un file JSON
    output_file_path = f"{extraction_path}/{article_id}.json"
    os.makedirs(extraction_path, exist_ok=True)
    with open(output_file_path, 'w', encoding='utf-8') as json_file:
        json.dump(tabelle, json_file, ensure_ascii=False, indent=4)

    logging.info(f"Dati salvati in {output_file_path}")

In [43]:
#per testare su di un unico file

file_path = os.path.join(articoli_path, 'ar5iv_article_2305.08291.html')

dati_estratti = estrai_dati_da_file(file_path)
logging.info(dati_estratti)
# Salva i dati in JSON
salva_dati_in_json(dati_estratti, '0', extraction_path)

2024-10-15 16:46:17,723 - INFO - {}


Dati salvati in /content/drive/MyDrive/progettoIngDati/extraction/0.json


In [ ]:
#MAIN
# Itera attraverso i file nella cartella 'articoli' e estrae le tabelle, didascalie, note e riferimenti
total_tables = 0
total_captions = 0
total_footnotes = 0
total_references = 0

articolo_counter = 0

#questo é perché nel salvataggio dei file ho creato cartella articoli dentro a sources
articoli_path = "/content/drive/MyDrive/progettoIngDati/sources/articoli"

for filename in os.listdir(articoli_path):
    if filename.endswith('.html'):
        file_path = os.path.join(articoli_path, filename)
        article_id = filename.split('_')[2].replace('.html', '')  # Estrae l'ID dal nome del file (primo elemento prima del punto)

        articolo_counter += 1  # Incrementa il contatore
        logging.info(f"Estraendo dati dall'articolo {articolo_counter}: {filename}...")

        # Estrai i dati dal file HTML
        dati_estratti = estrai_dati_da_file(file_path)

        # Update statistics counters
        for table_data in dati_estratti.values():
            total_tables += 1
            if table_data["caption"]:
                total_captions += 1
            total_footnotes += len(table_data["footnotes"])
            total_references += len(table_data["references"])

        # Salva i dati in JSON
        salva_dati_in_json(dati_estratti, article_id, extraction_path)

# Print statistics
logging.info("\n--- Extraction Statistics ---")
logging.info(f"Total Articles Processed: {articolo_counter}")
logging.info(f"Total Tables Found: {total_tables}")
logging.info(f"Total Captions Found: {total_captions}")
logging.info(f"Total Footnotes Found: {total_footnotes}")
logging.info(f"Total References Found: {total_references}")

2024-10-15 14:13:43,262 - INFO - Estraendo dati dall'articolo 1: ar5iv_article_2208.13266.html...
2024-10-15 14:13:43,475 - INFO - Footnotes for table id_table_1: []
2024-10-15 14:13:43,476 - INFO - Footnotes for table id_table_2: []
2024-10-15 14:13:43,479 - INFO - Footnotes for table id_table_3: []
2024-10-15 14:13:43,481 - INFO - Footnotes for table id_table_4: []
2024-10-15 14:13:43,483 - INFO - Footnotes for table id_table_5: []
2024-10-15 14:13:43,486 - INFO - Footnotes for table id_table_6: []
2024-10-15 14:13:43,487 - INFO - Footnotes for table id_table_7: []
2024-10-15 14:13:43,488 - INFO - Footnotes for table id_table_8: []
2024-10-15 14:13:43,489 - INFO - Footnotes for table id_table_9: []
2024-10-15 14:13:43,540 - INFO - Estraendo dati dall'articolo 2: ar5iv_article_2305.08291.html...
2024-10-15 14:13:43,590 - INFO - Estraendo dati dall'articolo 3: ar5iv_article_2306.03314.html...
2024-10-15 14:13:43,614 - INFO - Estraendo dati dall'articolo 4: ar5iv_article_2306.03553.html

Dati salvati in /content/drive/MyDrive/progettoIngDati/extraction/2208.13266.json
Dati salvati in /content/drive/MyDrive/progettoIngDati/extraction/2305.08291.json
Dati salvati in /content/drive/MyDrive/progettoIngDati/extraction/2306.03314.json
Dati salvati in /content/drive/MyDrive/progettoIngDati/extraction/2306.03553.json
Dati salvati in /content/drive/MyDrive/progettoIngDati/extraction/2306.04441.json


2024-10-15 14:13:43,755 - INFO - Estraendo dati dall'articolo 7: ar5iv_article_2307.02757.html...
2024-10-15 14:13:43,771 - INFO - Footnotes for table id_table_1: []
2024-10-15 14:13:43,802 - INFO - Estraendo dati dall'articolo 8: ar5iv_article_2307.04738.html...
2024-10-15 14:13:43,833 - INFO - Footnotes for table id_table_1: []
2024-10-15 14:13:43,834 - INFO - Footnotes for table id_table_2: []
2024-10-15 14:13:43,840 - INFO - Footnotes for table id_table_3: []
2024-10-15 14:13:43,841 - INFO - Footnotes for table id_table_4: []
2024-10-15 14:13:43,858 - INFO - Estraendo dati dall'articolo 9: ar5iv_article_2307.12573.html...
2024-10-15 14:13:43,877 - INFO - Footnotes for table id_table_1: []
2024-10-15 14:13:43,878 - INFO - Footnotes for table id_table_2: []
2024-10-15 14:13:43,895 - INFO - Estraendo dati dall'articolo 10: ar5iv_article_2308.05960.html...
2024-10-15 14:13:43,916 - INFO - Footnotes for table id_table_1: []
2024-10-15 14:13:43,917 - INFO - Footnotes for table id_table_2

Dati salvati in /content/drive/MyDrive/progettoIngDati/extraction/2307.01848.json
Dati salvati in /content/drive/MyDrive/progettoIngDati/extraction/2307.02757.json
Dati salvati in /content/drive/MyDrive/progettoIngDati/extraction/2307.04738.json
Dati salvati in /content/drive/MyDrive/progettoIngDati/extraction/2307.12573.json
Dati salvati in /content/drive/MyDrive/progettoIngDati/extraction/2308.05960.json


2024-10-15 14:13:43,965 - INFO - Footnotes for table id_table_1: []
2024-10-15 14:13:43,967 - INFO - Footnotes for table id_table_2: []
2024-10-15 14:13:43,971 - INFO - Footnotes for table id_table_3: []
2024-10-15 14:13:43,973 - INFO - Footnotes for table id_table_4: []
2024-10-15 14:13:43,975 - INFO - Footnotes for table id_table_5: []
2024-10-15 14:13:43,990 - INFO - Estraendo dati dall'articolo 12: ar5iv_article_2308.07201.html...
2024-10-15 14:13:44,035 - INFO - Footnotes for table id_table_1: []
2024-10-15 14:13:44,036 - INFO - Footnotes for table id_table_2: []
2024-10-15 14:13:44,039 - INFO - Footnotes for table id_table_3: []
2024-10-15 14:13:44,041 - INFO - Footnotes for table id_table_4: []
2024-10-15 14:13:44,043 - INFO - Footnotes for table id_table_5: []
2024-10-15 14:13:44,063 - INFO - Estraendo dati dall'articolo 13: ar5iv_article_2308.09853.html...
2024-10-15 14:13:44,080 - INFO - Footnotes for table id_table_1: []
2024-10-15 14:13:44,084 - INFO - Footnotes for table i

...

Dati salvati in /content/drive/MyDrive/progettoIngDati/extraction/2407.17115.json
Dati salvati in /content/drive/MyDrive/progettoIngDati/extraction/2407.17789.json


2024-10-15 14:14:34,520 - INFO - Estraendo dati dall'articolo 363: ar5iv_article_2407.18219.html...
2024-10-15 14:14:34,660 - INFO - Footnotes for table id_table_1: []
2024-10-15 14:14:34,661 - INFO - Footnotes for table id_table_2: []
2024-10-15 14:14:34,668 - INFO - Footnotes for table id_table_3: []
2024-10-15 14:14:34,672 - INFO - Footnotes for table id_table_4: []
2024-10-15 14:14:34,675 - INFO - Footnotes for table id_table_5: []
2024-10-15 14:14:34,677 - INFO - Footnotes for table id_table_6: []
2024-10-15 14:14:34,680 - INFO - Footnotes for table id_table_7: []
2024-10-15 14:14:34,682 - INFO - Footnotes for table id_table_8: []
2024-10-15 14:14:34,719 - INFO - Estraendo dati dall'articolo 364: ar5iv_article_2407.20183.html...


Dati salvati in /content/drive/MyDrive/progettoIngDati/extraction/2407.01603.json
Dati salvati in /content/drive/MyDrive/progettoIngDati/extraction/2407.18219.json


2024-10-15 14:14:34,748 - INFO - Footnotes for table id_table_1: ['†']
2024-10-15 14:14:34,783 - INFO - 
--- Extraction Statistics ---
2024-10-15 14:14:34,788 - INFO - Total Articles Processed: 364
2024-10-15 14:14:34,791 - INFO - Total Tables Found: 1846
2024-10-15 14:14:34,793 - INFO - Total Captions Found: 1826
2024-10-15 14:14:34,795 - INFO - Total Footnotes Found: 2237
2024-10-15 14:14:34,797 - INFO - Total References Found: 2013


Dati salvati in /content/drive/MyDrive/progettoIngDati/extraction/2407.20183.json
